In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_predict

import lightgbm as lgb

from imblearn.over_sampling import SMOTE

from keras.models import Sequential
from keras.layers import Dense

import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignore ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [2]:
df = pd.read_excel('Dataset.xlsx')

## Clean Data

In [3]:
# Get the list of columns to convert to categorical
categorical_columns = df.select_dtypes(include='int64').columns.tolist()

# Convert the selected columns to categorical
df[categorical_columns] = df[categorical_columns].astype('category')

numeric_data = df.select_dtypes(include=[np.number])

# Calculate MAD for each column
mad = numeric_data.mad()

# Choose a threshold multiplier
k = 3

# Calculate the threshold value
threshold = k * mad

# Identify outliers
outliers = (np.abs(numeric_data - numeric_data.median()) > threshold)

# Apply logarithm to the specified columns
outlier_columns = ['CommissionSacrificePercentage', 'BonusCommissionPercentage']
for column in outlier_columns:
    df[column] = np.log1p(df[column])
    
df = df.drop(columns=['PropDate'])


## Feature Engineering

In [4]:
# Convert categorical columns to strings
df['WorkflowStatus'] = df['WorkflowStatus'].astype(str)
df['UWDecision'] = df['UWDecision'].astype(str)

df['Product'] = df['Product'].astype(str)
df['ProductGroup'] = df['ProductGroup'].astype(str)
df['ProductType'] = df['ProductType'].astype(str)

df['CommissionSacrificeType'] = df['CommissionSacrificeType'].astype(str)
df['RenewalSacrificeType'] = df['RenewalSacrificeType'].astype(str)

df['CommDateProvided'] = df['CommDateProvided'].astype(str)
df['FreeCover'] = df['FreeCover'].astype(str)

# Combined Features
df['Combined_Status1'] = df['WorkflowStatus'] + '_' + df['UWDecision']
df['Combined_Status2'] = df['Product'] + '_' + df['ProductGroup'] + '_' + df['ProductType']
df['Combined_Status3'] = df['CommissionSacrificeType'] + '_' + df['RenewalSacrificeType']
df['Combined_Status4'] = df['CommDateProvided'] + '_' + df['FreeCover']

# Initialise LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the combined status
df['WorkflowStatus_UWDecision'] = label_encoder.fit_transform(df['Combined_Status1'])
df['Product_ProductGroup_ProductType'] = label_encoder.fit_transform(df['Combined_Status2'])
df['CommissionSacrificeType_RenewalSacrificeType'] = label_encoder.fit_transform(df['Combined_Status3'])
df['CommDateProvided_FreeCover'] = label_encoder.fit_transform(df['Combined_Status4'])


# Drop unnecessary columns
df.drop(['WorkflowStatus', 'UWDecision', 'Combined_Status1'], axis=1, inplace=True)
df.drop(['Product', 'ProductGroup', 'ProductType', 'Combined_Status2'], axis=1, inplace=True)
df.drop(['CommissionSacrificeType', 'RenewalSacrificeType', 'Combined_Status3'], axis=1, inplace=True)
df.drop(['CommDateProvided', 'FreeCover', 'Combined_Status4'], axis=1, inplace=True)

categorical_columns = df.select_dtypes(include='int32').columns.tolist()
df[categorical_columns] = df[categorical_columns].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157263 entries, 0 to 157262
Data columns (total 17 columns):
 #   Column                                        Non-Null Count   Dtype   
---  ------                                        --------------   -----   
 0   Agency                                        157263 non-null  category
 1   Indexation                                    157263 non-null  category
 2   NoOfLives                                     157263 non-null  category
 3   PaymentFreq                                   157263 non-null  category
 4   ComissionSacrifice                            157263 non-null  category
 5   CommissionSacrificePercentage                 157263 non-null  float64 
 6   CommissionTerms                               157263 non-null  category
 7   Discount                                      157263 non-null  category
 8   BonusCommission                               157263 non-null  category
 9   BonusCommissionPercentage            

In [5]:
# Select specific columns for each iteration
feature_sets = [
    ['Agency', 'WorkflowStatus_UWDecision'],
    ['Agency', 'WorkflowStatus_UWDecision', 'CommDateProvided_FreeCover'],
    ['Agency', 'WorkflowStatus_UWDecision', 'CommDateProvided_FreeCover', 'Product_ProductGroup_ProductType'],
    ['Agency', 'WorkflowStatus_UWDecision', 'CommDateProvided_FreeCover', 'Product_ProductGroup_ProductType','CommissionSacrificePercentage'],
    ['Agency', 'WorkflowStatus_UWDecision', 'CommDateProvided_FreeCover', 'Product_ProductGroup_ProductType','CommissionSacrificePercentage','BonusCommissionPercentage'],
    ['Agency', 'WorkflowStatus_UWDecision', 'CommDateProvided_FreeCover', 'Product_ProductGroup_ProductType','CommissionSacrificePercentage','BonusCommissionPercentage','SignedDecReceived']
]

# Top-n Features

## Logistic Regression

#### 80/20 Split

In [10]:

# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data
    model = LogisticRegression()
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7439441596372622
Test AUC: 0.7476153746967774
Training Accuracy: 0.7439441596372623
Test Accuracy: 0.6758655772104409
---------------------------------------------
Iteration 2: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover
Training AUC: 0.7459253283225424
Test AUC: 0.7494802366784474
Training Accuracy: 0.7459253283225424
Test Accuracy: 0.6801259021397005
---------------------------------------------
Iteration 3: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType
Training AUC: 0.7434131247319294
Test AUC: 0.7469247200101298
Training Accuracy: 0.7434131247319294
Test Accuracy: 0.680539217244778
---------------------------------------------
Iteration 4: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType, CommissionSacrificePercentage
Training AUC: 0.74557300708727

#### Cross Validation

In [11]:

# List of k values for k-fold cross-validation
num_folds_list = [5, 10, 20]

# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Iterate through different numbers of folds
    for num_folds in num_folds_list:
        print(f"Number of Folds: {num_folds}")
        
        # Initialise k-fold cross-validation
        kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

        # Lists to store evaluation metrics for each fold
        fold_auc_scores = []
        fold_accuracy_scores = []

        # Perform k-fold cross-validation
        for train_index, val_index in kf.split(X_resampled):
            X_fold_train, y_fold_train = X_resampled[train_index], y_resampled[train_index]
            X_fold_val, y_fold_val = X_resampled[val_index], y_resampled[val_index]

            # Train the model on the fold training data
            model = LogisticRegression()
            model.fit(X_fold_train, y_fold_train)

            # Predict on the fold validation data
            y_fold_val_pred = model.predict(X_fold_val)

            # Calculate AUC and accuracy for the fold
            fold_auc = roc_auc_score(y_fold_val, y_fold_val_pred)
            fold_accuracy = accuracy_score(y_fold_val, y_fold_val_pred)

            fold_auc_scores.append(fold_auc)
            fold_accuracy_scores.append(fold_accuracy)

        # Calculate average AUC and accuracy for the current number of folds
        avg_auc = sum(fold_auc_scores) / num_folds
        avg_accuracy = sum(fold_accuracy_scores) / num_folds

        # Print average metrics for cross-validation
        print("Average AUC:", avg_auc)
        print("Average Accuracy:", avg_accuracy)
        print("---------------------------------------------")
        
        # Train the model on the entire training data
        model.fit(X_resampled, y_resampled)

        # Predict on the test data
        y_pred = model.predict(X_test_scaled)

        # Calculate Test AUC and Test Accuracy
        test_auc = roc_auc_score(y_test, y_pred)
        test_accuracy = accuracy_score(y_test, y_pred)

        # Print evaluation metrics for the test data
        print("Test AUC:", test_auc)
        print("Test Accuracy:", test_accuracy)
        print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Number of Folds: 5
Average AUC: 0.7439607778623281
Average Accuracy: 0.7439492701738503
---------------------------------------------
Test AUC: 0.7476153746967774
Test Accuracy: 0.6758655772104409
---------------------------------------------
Number of Folds: 10
Average AUC: 0.7439567060261033
Average Accuracy: 0.7439492712134637
---------------------------------------------
Test AUC: 0.7476153746967774
Test Accuracy: 0.6758655772104409
---------------------------------------------
Number of Folds: 20
Average AUC: 0.743958086062096
Average Accuracy: 0.743949330908799
---------------------------------------------
Test AUC: 0.7476153746967774
Test Accuracy: 0.6758655772104409
---------------------------------------------
Iteration 2: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover
Number of Folds: 5
Average AUC: 0.7459358774341734
Average Accuracy: 0.7459253315290604
-------------------------------

#### 70/30 Split

In [12]:
# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data
    model = LogisticRegression()
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7451973576713895
Test AUC: 0.7466472060133872
Training Accuracy: 0.7451973576713895
Test Accuracy: 0.6751308845036986
---------------------------------------------
Iteration 2: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover
Training AUC: 0.7471814383417754
Test AUC: 0.7486929085507982
Training Accuracy: 0.7471814383417754
Test Accuracy: 0.6795396256809173
---------------------------------------------
Iteration 3: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType
Training AUC: 0.7406981629747205
Test AUC: 0.7422477285249909
Training Accuracy: 0.7406981629747205
Test Accuracy: 0.6805782233620891
---------------------------------------------
Iteration 4: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType, CommissionSacrificePercentage
Training AUC: 0.7451448378889

#### Cross Validation

In [13]:
# List of k values for k-fold cross-validation
num_folds_list = [5, 10, 20]

# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Iterate through different numbers of folds
    for num_folds in num_folds_list:
        print(f"Number of Folds: {num_folds}")
        
        # Initialise k-fold cross-validation
        kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

        # Lists to store evaluation metrics for each fold
        fold_auc_scores = []
        fold_accuracy_scores = []

        # Perform k-fold cross-validation
        for train_index, val_index in kf.split(X_resampled):
            X_fold_train, y_fold_train = X_resampled[train_index], y_resampled[train_index]
            X_fold_val, y_fold_val = X_resampled[val_index], y_resampled[val_index]

            # Train the model on the fold training data
            model = LogisticRegression()
            model.fit(X_fold_train, y_fold_train)

            # Predict on the fold validation data
            y_fold_val_pred = model.predict(X_fold_val)

            # Calculate AUC and accuracy for the fold
            fold_auc = roc_auc_score(y_fold_val, y_fold_val_pred)
            fold_accuracy = accuracy_score(y_fold_val, y_fold_val_pred)

            fold_auc_scores.append(fold_auc)
            fold_accuracy_scores.append(fold_accuracy)

        # Calculate average AUC and accuracy for the current number of folds
        avg_auc = sum(fold_auc_scores) / num_folds
        avg_accuracy = sum(fold_accuracy_scores) / num_folds

        # Print average metrics for cross-validation
        print("Average AUC:", avg_auc)
        print("Average Accuracy:", avg_accuracy)
        print("---------------------------------------------")
        
        # Train the model on the entire training data
        model.fit(X_resampled, y_resampled)

        # Predict on the test data
        y_pred = model.predict(X_test_scaled)

        # Calculate Test AUC and Test Accuracy
        test_auc = roc_auc_score(y_test, y_pred)
        test_accuracy = accuracy_score(y_test, y_pred)

        # Print evaluation metrics for the test data
        print("Test AUC:", test_auc)
        print("Test Accuracy:", test_accuracy)
        print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Number of Folds: 5
Average AUC: 0.7451942961415332
Average Accuracy: 0.7451973662864031
---------------------------------------------
Test AUC: 0.7466472060133872
Test Accuracy: 0.6751308845036986
---------------------------------------------
Number of Folds: 10
Average AUC: 0.745195903338684
Average Accuracy: 0.7451973889281452
---------------------------------------------
Test AUC: 0.7466472060133872
Test Accuracy: 0.6751308845036986
---------------------------------------------
Number of Folds: 20
Average AUC: 0.7451858817533348
Average Accuracy: 0.7451973640625191
---------------------------------------------
Test AUC: 0.7466472060133872
Test Accuracy: 0.6751308845036986
---------------------------------------------
Iteration 2: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover
Number of Folds: 5
Average AUC: 0.7471784284954918
Average Accuracy: 0.7471814476381397
------------------------------

### Stochastic Gradient Descent

#### 80/20 Split

In [14]:
# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using SGDClassifier
    model = SGDClassifier(loss='log', random_state=42)
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7439441596372622
Test AUC: 0.7476153746967774
Training Accuracy: 0.7439441596372623
Test Accuracy: 0.6758655772104409
---------------------------------------------
Iteration 2: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover
Training AUC: 0.7459661771614142
Test AUC: 0.7494702561024588
Training Accuracy: 0.7459661771614142
Test Accuracy: 0.6800305217308364
---------------------------------------------
Iteration 3: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType
Training AUC: 0.7449194256653255
Test AUC: 0.7484026537797891
Training Accuracy: 0.7449194256653254
Test Accuracy: 0.6802848694878072
---------------------------------------------
Iteration 4: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType, CommissionSacrificePercentage
Training AUC: 0.7414013194174

#### Cross Validation

In [15]:
# List of k values for k-fold cross-validation
num_folds_list = [5, 10, 20]

# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using SGDClassifier
    model = SGDClassifier(loss='log', random_state=42)
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")
    
    # Iterate through different numbers of folds
    for num_folds in num_folds_list:
        print(f"Number of Folds: {num_folds}")
        
        # Initialise k-fold cross-validation
        kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

        # Lists to store evaluation metrics for each fold
        fold_train_auc_scores = []
        fold_test_auc_scores = []
        fold_train_accuracy_scores = []
        fold_test_accuracy_scores = []

        # Perform k-fold cross-validation
        for train_index, val_index in kf.split(X_resampled):
            X_fold_train, y_fold_train = X_resampled[train_index], y_resampled[train_index]
            X_fold_val, y_fold_val = X_resampled[val_index], y_resampled[val_index]

            # Train the model on the fold training data using SGDClassifier
            model = SGDClassifier(loss='log', random_state=42)
            model.fit(X_fold_train, y_fold_train)

            # Predict on the fold validation data
            y_fold_val_pred = model.predict(X_fold_val)
            y_fold_test_pred = model.predict(X_test_scaled)

            # Calculate Training AUC, Test AUC, Training Accuracy, and Test Accuracy for the fold
            fold_train_auc = roc_auc_score(y_fold_train, model.predict(X_fold_train))
            fold_test_auc = roc_auc_score(y_test, y_fold_test_pred)
            fold_train_accuracy = accuracy_score(y_fold_train, model.predict(X_fold_train))
            fold_test_accuracy = accuracy_score(y_test, y_fold_test_pred)

            fold_train_auc_scores.append(fold_train_auc)
            fold_test_auc_scores.append(fold_test_auc)
            fold_train_accuracy_scores.append(fold_train_accuracy)
            fold_test_accuracy_scores.append(fold_test_accuracy)

        # Calculate average metrics for the current iteration's folds
        avg_train_auc = sum(fold_train_auc_scores) / num_folds
        avg_test_auc = sum(fold_test_auc_scores) / num_folds
        avg_train_accuracy = sum(fold_train_accuracy_scores) / num_folds
        avg_test_accuracy = sum(fold_test_accuracy_scores) / num_folds

        # Print average metrics for cross-validation
        print("Average Training AUC ({} folds):".format(num_folds), avg_train_auc)
        print("Average Test AUC ({} folds):".format(num_folds), avg_test_auc)
        print("Average Training Accuracy ({} folds):".format(num_folds), avg_train_accuracy)
        print("Average Test Accuracy ({} folds):".format(num_folds), avg_test_accuracy)
        print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7439441596372622
Test AUC: 0.7476153746967774
Training Accuracy: 0.7439441596372623
Test Accuracy: 0.6758655772104409
---------------------------------------------
Number of Folds: 5
Average Training AUC (5 folds): 0.743932136668168
Average Test AUC (5 folds): 0.7475723454368806
Average Training Accuracy (5 folds): 0.743931394653722
Average Test Accuracy (5 folds): 0.6758465011286681
---------------------------------------------
Number of Folds: 10
Average Training AUC (10 folds): 0.742540189707563
Average Test AUC (10 folds): 0.7461322048597501
Average Training Accuracy (10 folds): 0.7425405434342828
Average Test Accuracy (10 folds): 0.6772136203223857
---------------------------------------------
Number of Folds: 20
Average Training AUC (20 folds): 0.7433994854811516
Average Test AUC (20 folds): 0.7470236356620001
Average Training Accuracy (20 folds): 0.7433986134154545
Average Test Accuracy (20 folds): 0.6

#### 70/30 Split

In [16]:
# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using SGDClassifier
    model = SGDClassifier(loss='log', random_state=42)
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7451915221400061
Test AUC: 0.7464831972321369
Training Accuracy: 0.7451915221400061
Test Accuracy: 0.6750884927616101
---------------------------------------------
Iteration 2: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover
Training AUC: 0.7471697672790084
Test AUC: 0.7487201409246443
Training Accuracy: 0.7471697672790084
Test Accuracy: 0.679582017423006
---------------------------------------------
Iteration 3: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType
Training AUC: 0.7321724516234449
Test AUC: 0.7344017139386356
Training Accuracy: 0.7321724516234448
Test Accuracy: 0.6835880370503826
---------------------------------------------
Iteration 4: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType, CommissionSacrificePercentage
Training AUC: 0.73661912653766

#### Cross Validation

In [17]:
# List of k values for k-fold cross-validation
num_folds_list = [5, 10, 20]

# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using SGDClassifier
    model = SGDClassifier(loss='log', random_state=42)
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")
    
    # Iterate through different numbers of folds
    for num_folds in num_folds_list:
        print(f"Number of Folds: {num_folds}")
        
        # Initialise k-fold cross-validation
        kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

        # Lists to store evaluation metrics for each fold
        fold_train_auc_scores = []
        fold_test_auc_scores = []
        fold_train_accuracy_scores = []
        fold_test_accuracy_scores = []

        # Perform k-fold cross-validation
        for train_index, val_index in kf.split(X_resampled):
            X_fold_train, y_fold_train = X_resampled[train_index], y_resampled[train_index]
            X_fold_val, y_fold_val = X_resampled[val_index], y_resampled[val_index]

            # Train the model on the fold training data using SGDClassifier
            model = SGDClassifier(loss='log', random_state=42)
            model.fit(X_fold_train, y_fold_train)

            # Predict on the fold validation data
            y_fold_val_pred = model.predict(X_fold_val)
            y_fold_test_pred = model.predict(X_test_scaled)

            # Calculate Training AUC, Test AUC, Training Accuracy, and Test Accuracy for the fold
            fold_train_auc = roc_auc_score(y_fold_train, model.predict(X_fold_train))
            fold_test_auc = roc_auc_score(y_test, y_fold_test_pred)
            fold_train_accuracy = accuracy_score(y_fold_train, model.predict(X_fold_train))
            fold_test_accuracy = accuracy_score(y_test, y_fold_test_pred)

            fold_train_auc_scores.append(fold_train_auc)
            fold_test_auc_scores.append(fold_test_auc)
            fold_train_accuracy_scores.append(fold_train_accuracy)
            fold_test_accuracy_scores.append(fold_test_accuracy)

        # Calculate average metrics for the current iteration's folds
        avg_train_auc = sum(fold_train_auc_scores) / num_folds
        avg_test_auc = sum(fold_test_auc_scores) / num_folds
        avg_train_accuracy = sum(fold_train_accuracy_scores) / num_folds
        avg_test_accuracy = sum(fold_test_accuracy_scores) / num_folds

        # Print average metrics for cross-validation
        print("Average Training AUC ({} folds):".format(num_folds), avg_train_auc)
        print("Average Test AUC ({} folds):".format(num_folds), avg_test_auc)
        print("Average Training Accuracy ({} folds):".format(num_folds), avg_train_accuracy)
        print("Average Test Accuracy ({} folds):".format(num_folds), avg_test_accuracy)
        print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7451915221400061
Test AUC: 0.7464831972321369
Training Accuracy: 0.7451915221400061
Test Accuracy: 0.6750884927616101
---------------------------------------------
Number of Folds: 5
Average Training AUC (5 folds): 0.7390157425824853
Average Test AUC (5 folds): 0.7412209240311606
Average Training Accuracy (5 folds): 0.7390248560880666
Average Test Accuracy (5 folds): 0.6795014731130375
---------------------------------------------
Number of Folds: 10
Average Training AUC (10 folds): 0.7438147539841916
Average Test AUC (10 folds): 0.7454144352746497
Average Training Accuracy (10 folds): 0.7438143406138545
Average Test Accuracy (10 folds): 0.6764535068568642
---------------------------------------------
Number of Folds: 20
Average Training AUC (20 folds): 0.7442956986605849
Average Test AUC (20 folds): 0.7456275501478904
Average Training Accuracy (20 folds): 0.7442956150448106
Average Test Accuracy (20 folds): 

### Decision Tree

#### 80/20 Split

In [18]:
# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using DecisionTreeClassifier
    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7885408794755009
Test AUC: 0.7485773621952256
Training Accuracy: 0.7885408794755009
Test Accuracy: 0.7295011604616412
---------------------------------------------
Iteration 2: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover
Training AUC: 0.7924674741120483
Test AUC: 0.749970360003565
Training Accuracy: 0.7924674741120483
Test Accuracy: 0.730232410262932
---------------------------------------------
Iteration 3: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType
Training AUC: 0.8157717366883847
Test AUC: 0.7372691791868846
Training Accuracy: 0.8157717366883847
Test Accuracy: 0.7377674625631895
---------------------------------------------
Iteration 4: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType, CommissionSacrificePercentage
Training AUC: 0.832463593472355

#### Cross-Validation

In [19]:
# List of k values for k-fold cross-validation
num_folds_list = [5, 10, 20]

# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using DecisionTreeClassifier
    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")
    
    # Iterate through different numbers of folds
    for num_folds in num_folds_list:
        print(f"Number of Folds: {num_folds}")
        
        # Initialise k-fold cross-validation
        kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

        # Lists to store evaluation metrics for each fold
        fold_train_auc_scores = []
        fold_test_auc_scores = []
        fold_train_accuracy_scores = []
        fold_test_accuracy_scores = []

        # Perform k-fold cross-validation
        for train_index, val_index in kf.split(X_resampled):
            X_fold_train, y_fold_train = X_resampled[train_index], y_resampled[train_index]
            X_fold_val, y_fold_val = X_resampled[val_index], y_resampled[val_index]

            # Train the model on the fold training data using DecisionTreeClassifier
            model = DecisionTreeClassifier(random_state=42)
            model.fit(X_fold_train, y_fold_train)

            # Predict on the fold validation data
            y_fold_val_pred = model.predict(X_fold_val)
            y_fold_test_pred = model.predict(X_test_scaled)

            # Calculate Training AUC, Test AUC, Training Accuracy, and Test Accuracy for the fold
            fold_train_auc = roc_auc_score(y_fold_train, model.predict(X_fold_train))
            fold_test_auc = roc_auc_score(y_test, y_fold_test_pred)
            fold_train_accuracy = accuracy_score(y_fold_train, model.predict(X_fold_train))
            fold_test_accuracy = accuracy_score(y_test, y_fold_test_pred)

            fold_train_auc_scores.append(fold_train_auc)
            fold_test_auc_scores.append(fold_test_auc)
            fold_train_accuracy_scores.append(fold_train_accuracy)
            fold_test_accuracy_scores.append(fold_test_accuracy)

        # Calculate average metrics for the current iteration's folds
        avg_train_auc = sum(fold_train_auc_scores) / num_folds
        avg_test_auc = sum(fold_test_auc_scores) / num_folds
        avg_train_accuracy = sum(fold_train_accuracy_scores) / num_folds
        avg_test_accuracy = sum(fold_test_accuracy_scores) / num_folds

        # Print average metrics for cross-validation
        print("Average Training AUC ({} folds):".format(num_folds), avg_train_auc)
        print("Average Test AUC ({} folds):".format(num_folds), avg_test_auc)
        print("Average Training Accuracy ({} folds):".format(num_folds), avg_train_accuracy)
        print("Average Test Accuracy ({} folds):".format(num_folds), avg_test_accuracy)
        print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7885408794755009
Test AUC: 0.7485773621952256
Training Accuracy: 0.7885408794755009
Test Accuracy: 0.7295011604616412
---------------------------------------------
Number of Folds: 5
Average Training AUC (5 folds): 0.7894537148505918
Average Test AUC (5 folds): 0.7483430357304268
Average Training Accuracy (5 folds): 0.7894535955107211
Average Test Accuracy (5 folds): 0.7282421390646362
---------------------------------------------
Number of Folds: 10
Average Training AUC (10 folds): 0.7889545377405642
Average Test AUC (10 folds): 0.7480283730989712
Average Training Accuracy (10 folds): 0.7889544737395555
Average Test Accuracy (10 folds): 0.7285028455155311
---------------------------------------------
Number of Folds: 20
Average Training AUC (20 folds): 0.7887415750429241
Average Test AUC (20 folds): 0.7483579299897544
Average Training Accuracy (20 folds): 0.7887416300268568
Average Test Accuracy (20 folds): 

#### 70/30 Split

In [20]:
# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using DecisionTreeClassifier
    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7925585303797763
Test AUC: 0.7435855364670104
Training Accuracy: 0.7925585303797764
Test Accuracy: 0.7286504588906081
---------------------------------------------
Iteration 2: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover
Training AUC: 0.7964391587497958
Test AUC: 0.7446225175567698
Training Accuracy: 0.7964391587497958
Test Accuracy: 0.7295194896034253
---------------------------------------------
Iteration 3: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType
Training AUC: 0.8198162974720478
Test AUC: 0.7345028979094447
Training Accuracy: 0.8198162974720478
Test Accuracy: 0.7359630344008987
---------------------------------------------
Iteration 4: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType, CommissionSacrificePercentage
Training AUC: 0.8370486216474

#### Cross Validation

In [21]:
# List of k values for k-fold cross-validation
num_folds_list = [5, 10, 20]

# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using DecisionTreeClassifier
    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")
    
    # Iterate through different numbers of folds
    for num_folds in num_folds_list:
        print(f"Number of Folds: {num_folds}")
        
        # Initialise k-fold cross-validation
        kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

        # Lists to store evaluation metrics for each fold
        fold_train_auc_scores = []
        fold_test_auc_scores = []
        fold_train_accuracy_scores = []
        fold_test_accuracy_scores = []

        # Perform k-fold cross-validation
        for train_index, val_index in kf.split(X_resampled):
            X_fold_train, y_fold_train = X_resampled[train_index], y_resampled[train_index]
            X_fold_val, y_fold_val = X_resampled[val_index], y_resampled[val_index]

            # Train the model on the fold training data using DecisionTreeClassifier
            model = DecisionTreeClassifier(random_state=42)
            model.fit(X_fold_train, y_fold_train)

            # Predict on the fold validation data
            y_fold_val_pred = model.predict(X_fold_val)
            y_fold_test_pred = model.predict(X_test_scaled)

            # Calculate Training AUC, Test AUC, Training Accuracy, and Test Accuracy for the fold
            fold_train_auc = roc_auc_score(y_fold_train, model.predict(X_fold_train))
            fold_test_auc = roc_auc_score(y_test, y_fold_test_pred)
            fold_train_accuracy = accuracy_score(y_fold_train, model.predict(X_fold_train))
            fold_test_accuracy = accuracy_score(y_test, y_fold_test_pred)

            fold_train_auc_scores.append(fold_train_auc)
            fold_test_auc_scores.append(fold_test_auc)
            fold_train_accuracy_scores.append(fold_train_accuracy)
            fold_test_accuracy_scores.append(fold_test_accuracy)

        # Calculate average metrics for the current iteration's folds
        avg_train_auc = sum(fold_train_auc_scores) / num_folds
        avg_test_auc = sum(fold_test_auc_scores) / num_folds
        avg_train_accuracy = sum(fold_train_accuracy_scores) / num_folds
        avg_test_accuracy = sum(fold_test_accuracy_scores) / num_folds

        # Print average metrics for cross-validation
        print("Average Training AUC ({} folds):".format(num_folds), avg_train_auc)
        print("Average Test AUC ({} folds):".format(num_folds), avg_test_auc)
        print("Average Training Accuracy ({} folds):".format(num_folds), avg_train_accuracy)
        print("Average Test Accuracy ({} folds):".format(num_folds), avg_test_accuracy)
        print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7925585303797763
Test AUC: 0.7435855364670104
Training Accuracy: 0.7925585303797764
Test Accuracy: 0.7286504588906081
---------------------------------------------
Number of Folds: 5
Average Training AUC (5 folds): 0.7934430067624512
Average Test AUC (5 folds): 0.7433676582068248
Average Training Accuracy (5 folds): 0.7934440718717212
Average Test Accuracy (5 folds): 0.72737446745374
---------------------------------------------
Number of Folds: 10
Average Training AUC (10 folds): 0.7929573977816702
Average Test AUC (10 folds): 0.7435474936896435
Average Training Accuracy (10 folds): 0.7929579400114797
Average Test Accuracy (10 folds): 0.7280908878950381
---------------------------------------------
Number of Folds: 20
Average Training AUC (20 folds): 0.7927513315006736
Average Test AUC (20 folds): 0.7434178835525052
Average Training Accuracy (20 folds): 0.792751410045147
Average Test Accuracy (20 folds): 0.7

### Random Forest

#### 80/20 Split

In [22]:
# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using RandomForestClassifier
    model = RandomForestClassifier(random_state=42)
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7885204550560649
Test AUC: 0.7477225919204897
Training Accuracy: 0.788520455056065
Test Accuracy: 0.7306457253680094
---------------------------------------------
Iteration 2: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover
Training AUC: 0.7924317313780356
Test AUC: 0.7488182845698854
Training Accuracy: 0.7924317313780356
Test Accuracy: 0.7313133882300575
---------------------------------------------
Iteration 3: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType
Training AUC: 0.8157513122689487
Test AUC: 0.7369431187200735
Training Accuracy: 0.8157513122689487
Test Accuracy: 0.7405334944202461
---------------------------------------------
Iteration 4: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType, CommissionSacrificePercentage
Training AUC: 0.83244316905291

#### Cross Validation

In [23]:
# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using RandomForestClassifier
    model = RandomForestClassifier(random_state=42)
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

    # Iterate through different numbers of folds
    for num_folds in num_folds_list:
        print(f"Number of Folds: {num_folds}")
        
        # Initialise k-fold cross-validation
        kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

        # Lists to store evaluation metrics for each fold
        fold_train_auc_scores = []
        fold_test_auc_scores = []
        fold_train_accuracy_scores = []
        fold_test_accuracy_scores = []

        # Perform k-fold cross-validation
        for train_index, val_index in kf.split(X_resampled):
            X_fold_train, y_fold_train = X_resampled[train_index], y_resampled[train_index]
            X_fold_val, y_fold_val = X_resampled[val_index], y_resampled[val_index]

            # Train the model on the fold training data using RandomForestClassifier
            model = RandomForestClassifier(random_state=42)
            model.fit(X_fold_train, y_fold_train)

            # Predict on the fold validation data
            y_fold_val_pred = model.predict(X_fold_val)
            y_fold_test_pred = model.predict(X_test_scaled)

            # Calculate Training AUC, Test AUC, Training Accuracy, and Test Accuracy for the fold
            fold_train_auc = roc_auc_score(y_fold_train, model.predict(X_fold_train))
            fold_test_auc = roc_auc_score(y_test, y_fold_test_pred)
            fold_train_accuracy = accuracy_score(y_fold_train, model.predict(X_fold_train))
            fold_test_accuracy = accuracy_score(y_test, y_fold_test_pred)

            fold_train_auc_scores.append(fold_train_auc)
            fold_test_auc_scores.append(fold_test_auc)
            fold_train_accuracy_scores.append(fold_train_accuracy)
            fold_test_accuracy_scores.append(fold_test_accuracy)

        # Calculate average metrics for the current iteration's folds
        avg_train_auc = sum(fold_train_auc_scores) / num_folds
        avg_test_auc = sum(fold_test_auc_scores) / num_folds
        avg_train_accuracy = sum(fold_train_accuracy_scores) / num_folds
        avg_test_accuracy = sum(fold_test_accuracy_scores) / num_folds

        # Print average metrics for cross-validation
        print("Average Training AUC ({} folds):".format(num_folds), avg_train_auc)
        print("Average Test AUC ({} folds):".format(num_folds), avg_test_auc)
        print("Average Training Accuracy ({} folds):".format(num_folds), avg_train_accuracy)
        print("Average Test Accuracy ({} folds):".format(num_folds), avg_test_accuracy)
        print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7885204550560649
Test AUC: 0.7477225919204897
Training Accuracy: 0.788520455056065
Test Accuracy: 0.7306457253680094
---------------------------------------------
Number of Folds: 5
Average Training AUC (5 folds): 0.7894372291008462
Average Test AUC (5 folds): 0.7476448359978395
Average Training Accuracy (5 folds): 0.789437000681336
Average Test Accuracy (5 folds): 0.7298699647092487
---------------------------------------------
Number of Folds: 10
Average Training AUC (10 folds): 0.7889284112831405
Average Test AUC (10 folds): 0.7474144209111038
Average Training Accuracy (10 folds): 0.788928375868345
Average Test Accuracy (10 folds): 0.7300861603026739
---------------------------------------------
Number of Folds: 20
Average Training AUC (20 folds): 0.7887229983295444
Average Test AUC (20 folds): 0.7477813427114072
Average Training Accuracy (20 folds): 0.7887230868007702
Average Test Accuracy (20 folds): 0.7

#### 70/30 Split

In [24]:
# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using RandomForestClassifier
    model = RandomForestClassifier(random_state=42)
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7925293527228588
Test AUC: 0.7427118441593269
Training Accuracy: 0.792529352722859
Test Accuracy: 0.730430912058331
---------------------------------------------
Iteration 2: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover
Training AUC: 0.7964158166242619
Test AUC: 0.743777286699281
Training Accuracy: 0.7964158166242618
Test Accuracy: 0.7309184170923504
---------------------------------------------
Iteration 3: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType
Training AUC: 0.8197929553465138
Test AUC: 0.733335276680111
Training Accuracy: 0.8197929553465139
Test Accuracy: 0.7400538375124526
---------------------------------------------
Iteration 4: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType, CommissionSacrificePercentage
Training AUC: 0.8370311150533368


#### Cross Validation

In [7]:
# List of k values for k-fold cross-validation
num_folds_list = [5, 10, 20]

# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using RandomForestClassifier
    model = RandomForestClassifier(random_state=42)
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

    # Iterate through different numbers of folds
    for num_folds in num_folds_list:
        print(f"Number of Folds: {num_folds}")
        
        # Initialise k-fold cross-validation
        kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

        # Lists to store evaluation metrics for each fold
        fold_train_auc_scores = []
        fold_test_auc_scores = []
        fold_train_accuracy_scores = []
        fold_test_accuracy_scores = []

        # Perform k-fold cross-validation
        for train_index, val_index in kf.split(X_resampled):
            X_fold_train, y_fold_train = X_resampled[train_index], y_resampled[train_index]
            X_fold_val, y_fold_val = X_resampled[val_index], y_resampled[val_index]

            # Train the model on the fold training data using RandomForestClassifier
            model = RandomForestClassifier(random_state=42)
            model.fit(X_fold_train, y_fold_train)

            # Predict on the fold validation data
            y_fold_val_pred = model.predict(X_fold_val)
            y_fold_test_pred = model.predict(X_test_scaled)

            # Calculate Training AUC, Test AUC, Training Accuracy, and Test Accuracy for the fold
            fold_train_auc = roc_auc_score(y_fold_train, model.predict(X_fold_train))
            fold_test_auc = roc_auc_score(y_test, y_fold_test_pred)
            fold_train_accuracy = accuracy_score(y_fold_train, model.predict(X_fold_train))
            fold_test_accuracy = accuracy_score(y_test, y_fold_test_pred)

            fold_train_auc_scores.append(fold_train_auc)
            fold_test_auc_scores.append(fold_test_auc)
            fold_train_accuracy_scores.append(fold_train_accuracy)
            fold_test_accuracy_scores.append(fold_test_accuracy)

        # Calculate average metrics for the current iteration's folds
        avg_train_auc = sum(fold_train_auc_scores) / num_folds
        avg_test_auc = sum(fold_test_auc_scores) / num_folds
        avg_train_accuracy = sum(fold_train_accuracy_scores) / num_folds
        avg_test_accuracy = sum(fold_test_accuracy_scores) / num_folds

        # Print average metrics for cross-validation
        print("Average Training AUC ({} folds):".format(num_folds), avg_train_auc)
        print("Average Test AUC ({} folds):".format(num_folds), avg_test_auc)
        print("Average Training Accuracy ({} folds):".format(num_folds), avg_train_accuracy)
        print("Average Test Accuracy ({} folds):".format(num_folds), avg_test_accuracy)
        print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7925293527228588
Test AUC: 0.7427118441593269
Training Accuracy: 0.792529352722859
Test Accuracy: 0.730430912058331
---------------------------------------------
Number of Folds: 5
Average Training AUC (5 folds): 0.7934330107667
Average Test AUC (5 folds): 0.742440586227963
Average Training Accuracy (5 folds): 0.7934338596769017
Average Test Accuracy (5 folds): 0.7291676381440895
---------------------------------------------
Number of Folds: 10
Average Training AUC (10 folds): 0.7929355203335834
Average Test AUC (10 folds): 0.7426240360285191
Average Training Accuracy (10 folds): 0.7929358946690157
Average Test Accuracy (10 folds): 0.7295809576294537
---------------------------------------------
Number of Folds: 20
Average Training AUC (20 folds): 0.79273139753433
Average Test AUC (20 folds): 0.7426065702546889
Average Training Accuracy (20 folds): 0.7927314463813776
Average Test Accuracy (20 folds): 0.729708

### LightGBM

#### 80/20 Split

In [8]:
# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using LGBMClassifier
    model = lgb.LGBMClassifier(random_state=42)
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7518586221686648
Test AUC: 0.7535859825771895
Training Accuracy: 0.7518586221686648
Test Accuracy: 0.6933837789717991
---------------------------------------------
Iteration 2: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover
Training AUC: 0.7541002022017526
Test AUC: 0.7556924319146607
Training Accuracy: 0.7541002022017524
Test Accuracy: 0.695704702254157
---------------------------------------------
Iteration 3: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType
Training AUC: 0.7544576295418802
Test AUC: 0.7552965074159282
Training Accuracy: 0.7544576295418802
Test Accuracy: 0.6978030712491654
---------------------------------------------
Iteration 4: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType, CommissionSacrificePercentage
Training AUC: 0.75523886358530

#### Cross Validation

In [10]:
# List of k values for k-fold cross-validation
num_folds_list = [5, 10, 20]

# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using LGBMClassifier
    model = lgb.LGBMClassifier(random_state=42)
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

    # Iterate through different numbers of folds
    for num_folds in num_folds_list:
        print(f"Number of Folds: {num_folds}")
        
        # Initialise k-fold cross-validation
        kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

        # Lists to store evaluation metrics for each fold
        fold_train_auc_scores = []
        fold_test_auc_scores = []
        fold_train_accuracy_scores = []
        fold_test_accuracy_scores = []

        # Perform k-fold cross-validation
        for train_index, val_index in kf.split(X_resampled):
            X_fold_train, y_fold_train = X_resampled[train_index], y_resampled[train_index]
            X_fold_val, y_fold_val = X_resampled[val_index], y_resampled[val_index]

            # Train the model on the fold training data using LGBMClassifier
            model = lgb.LGBMClassifier(random_state=42)
            model.fit(X_fold_train, y_fold_train)

            # Predict on the fold validation data
            y_fold_val_pred = model.predict(X_fold_val)
            y_fold_test_pred = model.predict(X_test_scaled)

            # Calculate Training AUC, Test AUC, Training Accuracy, and Test Accuracy for the fold
            fold_train_auc = roc_auc_score(y_fold_train, model.predict(X_fold_train))
            fold_test_auc = roc_auc_score(y_test, y_fold_test_pred)
            fold_train_accuracy = accuracy_score(y_fold_train, model.predict(X_fold_train))
            fold_test_accuracy = accuracy_score(y_test, y_fold_test_pred)

            fold_train_auc_scores.append(fold_train_auc)
            fold_test_auc_scores.append(fold_test_auc)
            fold_train_accuracy_scores.append(fold_train_accuracy)
            fold_test_accuracy_scores.append(fold_test_accuracy)

        # Calculate average metrics for the current iteration's folds
        avg_train_auc = sum(fold_train_auc_scores) / num_folds
        avg_test_auc = sum(fold_test_auc_scores) / num_folds
        avg_train_accuracy = sum(fold_train_accuracy_scores) / num_folds
        avg_test_accuracy = sum(fold_test_accuracy_scores) / num_folds

        # Print average metrics for cross-validation
        print("Average Training AUC ({} folds):".format(num_folds), avg_train_auc)
        print("Average Test AUC ({} folds):".format(num_folds), avg_test_auc)
        print("Average Training Accuracy ({} folds):".format(num_folds), avg_train_accuracy)
        print("Average Test Accuracy ({} folds):".format(num_folds), avg_test_accuracy)
        print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7518586221686648
Test AUC: 0.7535859825771895
Training Accuracy: 0.7518586221686648
Test Accuracy: 0.6933837789717991
---------------------------------------------
Number of Folds: 5
Average Training AUC (5 folds): 0.7517830424163694
Average Test AUC (5 folds): 0.7535425426108785
Average Training Accuracy (5 folds): 0.7517820308499958
Average Test Accuracy (5 folds): 0.6942104091819539
---------------------------------------------
Number of Folds: 10
Average Training AUC (10 folds): 0.7518377237322033
Average Test AUC (10 folds): 0.7537167685488082
Average Training Accuracy (10 folds): 0.7518376303379514
Average Test Accuracy (10 folds): 0.6939147299144756
---------------------------------------------
Number of Folds: 20
Average Training AUC (20 folds): 0.7517013523649454
Average Test AUC (20 folds): 0.7535583638890703
Average Training Accuracy (20 folds): 0.7517014081342014
Average Test Accuracy (20 folds): 

#### 70/30 Split

In [11]:
# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using LGBMClassifier
    model = lgb.LGBMClassifier(random_state=42)
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7531570224784667
Test AUC: 0.7522009593576273
Training Accuracy: 0.7531570224784669
Test Accuracy: 0.6937832510227008
---------------------------------------------
Iteration 2: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover
Training AUC: 0.755251978245139
Test AUC: 0.7539868307281523
Training Accuracy: 0.755251978245139
Test Accuracy: 0.6958180546429555
---------------------------------------------
Iteration 3: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType
Training AUC: 0.7567983940617633
Test AUC: 0.7536768274668816
Training Accuracy: 0.7567983940617633
Test Accuracy: 0.698955043557515
---------------------------------------------
Iteration 4: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover, Product_ProductGroup_ProductType, CommissionSacrificePercentage
Training AUC: 0.7567983940617633

#### Cross Validation

In [12]:
# List of k values for k-fold cross-validation
num_folds_list = [5, 10, 20]

# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using LGBMClassifier
    model = lgb.LGBMClassifier(random_state=42)
    model.fit(X_resampled, y_resampled)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

    # Iterate through different numbers of folds
    for num_folds in num_folds_list:
        print(f"Number of Folds: {num_folds}")
        
        # Initialise k-fold cross-validation
        kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

        # Lists to store evaluation metrics for each fold
        fold_train_auc_scores = []
        fold_test_auc_scores = []
        fold_train_accuracy_scores = []
        fold_test_accuracy_scores = []

        # Perform k-fold cross-validation
        for train_index, val_index in kf.split(X_resampled):
            X_fold_train, y_fold_train = X_resampled[train_index], y_resampled[train_index]
            X_fold_val, y_fold_val = X_resampled[val_index], y_resampled[val_index]

            # Train the model on the fold training data using LGBMClassifier
            model = lgb.LGBMClassifier(random_state=42)
            model.fit(X_fold_train, y_fold_train)

            # Predict on the fold validation data
            y_fold_val_pred = model.predict(X_fold_val)
            y_fold_test_pred = model.predict(X_test_scaled)

            # Calculate Training AUC, Test AUC, Training Accuracy, and Test Accuracy for the fold
            fold_train_auc = roc_auc_score(y_fold_train, model.predict(X_fold_train))
            fold_test_auc = roc_auc_score(y_test, y_fold_test_pred)
            fold_train_accuracy = accuracy_score(y_fold_train, model.predict(X_fold_train))
            fold_test_accuracy = accuracy_score(y_test, y_fold_test_pred)

            fold_train_auc_scores.append(fold_train_auc)
            fold_test_auc_scores.append(fold_test_auc)
            fold_train_accuracy_scores.append(fold_train_accuracy)
            fold_test_accuracy_scores.append(fold_test_accuracy)

        # Calculate average metrics for the current iteration's folds
        avg_train_auc = sum(fold_train_auc_scores) / num_folds
        avg_test_auc = sum(fold_test_auc_scores) / num_folds
        avg_train_accuracy = sum(fold_train_accuracy_scores) / num_folds
        avg_test_accuracy = sum(fold_test_accuracy_scores) / num_folds

        # Print average metrics for cross-validation
        print("Average Training AUC ({} folds):".format(num_folds), avg_train_auc)
        print("Average Test AUC ({} folds):".format(num_folds), avg_test_auc)
        print("Average Training Accuracy ({} folds):".format(num_folds), avg_train_accuracy)
        print("Average Test Accuracy ({} folds):".format(num_folds), avg_test_accuracy)
        print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Training AUC: 0.7531570224784667
Test AUC: 0.7522009593576273
Training Accuracy: 0.7531570224784669
Test Accuracy: 0.6937832510227008
---------------------------------------------
Number of Folds: 5
Average Training AUC (5 folds): 0.7532448490550101
Average Test AUC (5 folds): 0.7522501427225851
Average Training Accuracy (5 folds): 0.7532445563303753
Average Test Accuracy (5 folds): 0.6928591110451684
---------------------------------------------
Number of Folds: 10
Average Training AUC (10 folds): 0.7531546364644578
Average Test AUC (10 folds): 0.752056813049087
Average Training Accuracy (10 folds): 0.7531550775855702
Average Test Accuracy (10 folds): 0.6927870450836178
---------------------------------------------
Number of Folds: 20
Average Training AUC (20 folds): 0.753289862979946
Average Test AUC (20 folds): 0.7519511949640115
Average Training Accuracy (20 folds): 0.753289704024343
Average Test Accuracy (20 folds): 0.6

## Neural Network

#### 8020 Split

Loop over epochs

In [13]:
# List of epochs to iterate over
epochs_list = [10, 20, 30]

# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using a neural network
    model = Sequential()
    model.add(Dense(units=64, activation='relu', input_dim=len(features)))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Loop over epochs
    for epochs in epochs_list:
        print(f"Epochs: {epochs}")
        
        model.fit(X_resampled, y_resampled, epochs=epochs, batch_size=32, verbose=0)

        # Predict on the test data
        y_pred_train = np.round(model.predict(X_resampled)).astype(int)
        y_pred_test = np.round(model.predict(X_test_scaled)).astype(int)

        # Calculate Training AUC and Test AUC
        train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
        test_auc = roc_auc_score(y_test, y_pred_test)

        # Calculate Training Accuracy and Test Accuracy
        train_accuracy = accuracy_score(y_resampled, y_pred_train)
        test_accuracy = accuracy_score(y_test, y_pred_test)

        # Print evaluation metrics for training and test data
        print("Training AUC:", train_auc)
        print("Test AUC:", test_auc)
        print("Training Accuracy:", train_accuracy)
        print("Test Accuracy:", test_accuracy)
        print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Epochs: 10
6121/6121 [==============================] - 7s 1ms/step
Training AUC: 0.7838767211273034
Test AUC: 0.7477179575585248
Training Accuracy: 0.7438982046935316
Test Accuracy: 0.6758655772104409
---------------------------------------------
Epochs: 20
6121/6121 [==============================] - 5s 818us/step
Training AUC: 0.7871488863661955
Test AUC: 0.74751232898472
Training Accuracy: 0.7440513878393007
Test Accuracy: 0.6760245445585477
---------------------------------------------
Epochs: 30
6121/6121 [==============================] - 5s 852us/step
Training AUC: 0.787891443550339
Test AUC: 0.7478855152296148
Training Accuracy: 0.7439135230081085
Test Accuracy: 0.6746892188344514
---------------------------------------------
Iteration 2: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover
Epochs: 10
6121/6121 [==============================] - 5s 797us/step
Training AUC: 0.7909614436527634


#### 70/30 Split

In [14]:
# Iterate through feature sets
for i, features in enumerate(feature_sets, start=1):
    print(f"Iteration {i}: Using features {', '.join(features)}")
    
    # Extract the selected features and target variable
    X = df[features]
    y = df["PolicyIssued"]

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Convert the resampled arrays back to a dataframe
    resampled_df = pd.DataFrame(X_resampled, columns=features)
    resampled_df["PolicyIssued"] = y_resampled

    # Train the model on the entire training data using a neural network
    model = Sequential()
    model.add(Dense(units=64, activation='relu', input_dim=len(features)))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Loop over epochs
    for epochs in epochs_list:
        print(f"Epochs: {epochs}")
        
        model.fit(X_resampled, y_resampled, epochs=epochs, batch_size=32, verbose=0)

        # Predict on the test data
        y_pred_train = np.round(model.predict(X_resampled)).astype(int)
        y_pred_test = np.round(model.predict(X_test_scaled)).astype(int)

        # Calculate Training AUC and Test AUC
        train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
        test_auc = roc_auc_score(y_test, y_pred_test)

        # Calculate Training Accuracy and Test Accuracy
        train_accuracy = accuracy_score(y_resampled, y_pred_train)
        test_accuracy = accuracy_score(y_test, y_pred_test)

        # Print evaluation metrics for training and test data
        print("Training AUC:", train_auc)
        print("Test AUC:", test_auc)
        print("Training Accuracy:", train_accuracy)
        print("Test Accuracy:", test_accuracy)
        print("---------------------------------------------")

Iteration 1: Using features Agency, WorkflowStatus_UWDecision
Epochs: 10
5356/5356 [==============================] - 4s 794us/step
Training AUC: 0.7842579803528085
Test AUC: 0.7469794107681952
Training Accuracy: 0.7448122126000793
Test Accuracy: 0.6832065113715848
---------------------------------------------
Epochs: 20
5356/5356 [==============================] - 5s 841us/step
Training AUC: 0.7873955744660521
Test AUC: 0.7467089397581717
Training Accuracy: 0.7451856866086226
Test Accuracy: 0.6750672968905658
---------------------------------------------
Epochs: 30
5356/5356 [==============================] - 6s 1ms/step
Training AUC: 0.788850135932098
Test AUC: 0.7470892711847544
Training Accuracy: 0.7454016012698116
Test Accuracy: 0.6759787193454715
---------------------------------------------
Iteration 2: Using features Agency, WorkflowStatus_UWDecision, CommDateProvided_FreeCover
Epochs: 10
5356/5356 [==============================] - 5s 839us/step
Training AUC: 0.791369132321432

### All Features

In [24]:
df = pd.read_excel('Dataset.xlsx')

# Get the list of columns to convert to categorical
categorical_columns = df.select_dtypes(include='int64').columns.tolist()

# Convert the selected columns to categorical
df[categorical_columns] = df[categorical_columns].astype('category')

numeric_data = df.select_dtypes(include=[np.number])

# Calculate MAD for each column
mad = numeric_data.mad()

# Choose a threshold multiplier
k = 3

# Calculate the threshold value
threshold = k * mad

# Identify outliers
outliers = (np.abs(numeric_data - numeric_data.median()) > threshold)

# Apply logarithm to the specified columns
outlier_columns = ['CommissionSacrificePercentage', 'BonusCommissionPercentage']
for column in outlier_columns:
    df[column] = np.log1p(df[column])
    
df = df.drop(columns=['PropDate'])



In [25]:
# Convert categorical columns to strings
df['WorkflowStatus'] = df['WorkflowStatus'].astype(str)
df['UWDecision'] = df['UWDecision'].astype(str)

df['Product'] = df['Product'].astype(str)
df['ProductGroup'] = df['ProductGroup'].astype(str)
df['ProductType'] = df['ProductType'].astype(str)

df['CommissionSacrificeType'] = df['CommissionSacrificeType'].astype(str)
df['RenewalSacrificeType'] = df['RenewalSacrificeType'].astype(str)

df['CommDateProvided'] = df['CommDateProvided'].astype(str)
df['FreeCover'] = df['FreeCover'].astype(str)

# Combined Features
df['Combined_Status1'] = df['WorkflowStatus'] + '_' + df['UWDecision']
df['Combined_Status2'] = df['Product'] + '_' + df['ProductGroup'] + '_' + df['ProductType']
df['Combined_Status3'] = df['CommissionSacrificeType'] + '_' + df['RenewalSacrificeType']
df['Combined_Status4'] = df['CommDateProvided'] + '_' + df['FreeCover']

# Initialise LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the combined status
df['WorkflowStatus_UWDecision'] = label_encoder.fit_transform(df['Combined_Status1'])
df['Product_ProductGroup_ProductType'] = label_encoder.fit_transform(df['Combined_Status2'])
df['CommissionSacrificeType_RenewalSacrificeType'] = label_encoder.fit_transform(df['Combined_Status3'])
df['CommDateProvided_FreeCover'] = label_encoder.fit_transform(df['Combined_Status4'])


# Drop unnecessary columns
df.drop(['WorkflowStatus', 'UWDecision', 'Combined_Status1'], axis=1, inplace=True)
df.drop(['Product', 'ProductGroup', 'ProductType', 'Combined_Status2'], axis=1, inplace=True)
df.drop(['CommissionSacrificeType', 'RenewalSacrificeType', 'Combined_Status3'], axis=1, inplace=True)
df.drop(['CommDateProvided', 'FreeCover', 'Combined_Status4'], axis=1, inplace=True)

categorical_columns = df.select_dtypes(include='int32').columns.tolist()
df[categorical_columns] = df[categorical_columns].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157263 entries, 0 to 157262
Data columns (total 17 columns):
 #   Column                                        Non-Null Count   Dtype   
---  ------                                        --------------   -----   
 0   Agency                                        157263 non-null  category
 1   Indexation                                    157263 non-null  category
 2   NoOfLives                                     157263 non-null  category
 3   PaymentFreq                                   157263 non-null  category
 4   ComissionSacrifice                            157263 non-null  category
 5   CommissionSacrificePercentage                 157263 non-null  float64 
 6   CommissionTerms                               157263 non-null  category
 7   Discount                                      157263 non-null  category
 8   BonusCommission                               157263 non-null  category
 9   BonusCommissionPercentage            

### Logistic Regression

#### 80/20 Split

In [29]:
# Extract the selected features and target variable
X = df.drop('PolicyIssued', axis=1)
y = df["PolicyIssued"]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Applying SMOTE to the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

# Convert the resampled arrays back to a dataframe
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df["PolicyIssued"] = y_resampled

# Train the model on the entire training data
model = LogisticRegression()
model.fit(X_resampled, y_resampled)

# Predict on the test data
y_pred_test = model.predict(X_test_scaled)

# Calculate Training AUC and Test AUC
train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
test_auc = roc_auc_score(y_test, y_pred_test)

# Calculate Training Accuracy and Test Accuracy
train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
test_accuracy = accuracy_score(y_test, y_pred_test)

# Print evaluation metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("---------------------------------------------")

Training AUC: 0.7228150977308471
Test AUC: 0.7237715150107746
Training Accuracy: 0.722815097730847
Test Accuracy: 0.6947191046958955
---------------------------------------------


#### Cross Validation

In [31]:
from sklearn.model_selection import cross_val_predict

# Define a list of k values for k-fold cross-validation
num_folds_list = [5, 10, 20]

# Loop over different values of k
for num_folds in num_folds_list:
    print(f"Performing {num_folds}-fold Cross-Validation")

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Create a logistic regression model
    model = LogisticRegression()

    # Perform k-fold cross-validation and get predicted probabilities
    predicted_probs = cross_val_predict(model, X_resampled, y_resampled, cv=num_folds, method='predict_proba')

    # Calculate AUC for training data
    train_auc = roc_auc_score(y_resampled, predicted_probs[:, 1])

    # Train the model on the entire training data
    model.fit(X_resampled, y_resampled)

    # Predict on the train data
    y_pred_train = model.predict(X_resampled)

    # Calculate training accuracy
    train_accuracy = accuracy_score(y_resampled, y_pred_train)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate AUC and accuracy for test data
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for each value of k
    print(f"{num_folds}-fold Cross-Validation")
    print("Training AUC:", train_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test AUC:", test_auc)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Performing 5-fold Cross-Validation
5-fold Cross-Validation
Training AUC: 0.7569870845000433
Training Accuracy: 0.722815097730847
Test AUC: 0.7237715150107746
Test Accuracy: 0.6947191046958955
---------------------------------------------
Performing 10-fold Cross-Validation
10-fold Cross-Validation
Training AUC: 0.7569569820924562
Training Accuracy: 0.722815097730847
Test AUC: 0.7237715150107746
Test Accuracy: 0.6947191046958955
---------------------------------------------
Performing 20-fold Cross-Validation
20-fold Cross-Validation
Training AUC: 0.7569711558849079
Training Accuracy: 0.722815097730847
Test AUC: 0.7237715150107746
Test Accuracy: 0.6947191046958955
---------------------------------------------


#### 70/30 Split

In [32]:
# Extract the selected features and target variable
X = df.drop('PolicyIssued', axis=1)
y = df["PolicyIssued"]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Applying SMOTE to the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

# Convert the resampled arrays back to a dataframe
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df["PolicyIssued"] = y_resampled

# Train the model on the entire training data
model = LogisticRegression()
model.fit(X_resampled, y_resampled)

# Predict on the test data
y_pred_test = model.predict(X_test_scaled)

# Calculate Training AUC and Test AUC
train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
test_auc = roc_auc_score(y_test, y_pred_test)

# Calculate Training Accuracy and Test Accuracy
train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
test_accuracy = accuracy_score(y_test, y_pred_test)

# Print evaluation metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("---------------------------------------------")

Training AUC: 0.7227247263135781
Test AUC: 0.7239563239595065
Training Accuracy: 0.7227247263135781
Test Accuracy: 0.6938892303779224
---------------------------------------------


In [33]:
from sklearn.model_selection import cross_val_predict

# Define a list of k values for k-fold cross-validation
num_folds_list = [5, 10, 20]

# Loop over different values of k
for num_folds in num_folds_list:
    print(f"Performing {num_folds}-fold Cross-Validation")

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Create a logistic regression model
    model = LogisticRegression()

    # Perform k-fold cross-validation and get predicted probabilities
    predicted_probs = cross_val_predict(model, X_resampled, y_resampled, cv=num_folds, method='predict_proba')

    # Calculate AUC for training data
    train_auc = roc_auc_score(y_resampled, predicted_probs[:, 1])

    # Train the model on the entire training data
    model.fit(X_resampled, y_resampled)

    # Predict on the train data
    y_pred_train = model.predict(X_resampled)

    # Calculate training accuracy
    train_accuracy = accuracy_score(y_resampled, y_pred_train)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate AUC and accuracy for test data
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for each value of k
    print(f"{num_folds}-fold Cross-Validation")
    print("Training AUC:", train_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test AUC:", test_auc)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Performing 5-fold Cross-Validation
5-fold Cross-Validation
Training AUC: 0.7570198739382361
Training Accuracy: 0.7227247263135781
Test AUC: 0.7239563239595065
Test Accuracy: 0.6938892303779224
---------------------------------------------
Performing 10-fold Cross-Validation
10-fold Cross-Validation
Training AUC: 0.7570035817573824
Training Accuracy: 0.7227247263135781
Test AUC: 0.7239563239595065
Test Accuracy: 0.6938892303779224
---------------------------------------------
Performing 20-fold Cross-Validation
20-fold Cross-Validation
Training AUC: 0.756998097997895
Training Accuracy: 0.7227247263135781
Test AUC: 0.7239563239595065
Test Accuracy: 0.6938892303779224
---------------------------------------------


### Stochastic Gradient Descent

#### 80/20 Split

In [38]:
# Extract the selected features and target variable
X = df.drop('PolicyIssued', axis=1)
y = df["PolicyIssued"]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Applying SMOTE to the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

# Convert the resampled arrays back to a dataframe
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df["PolicyIssued"] = y_resampled

# Train the model on the entire training data
model = SGDClassifier(loss='log', random_state=42)
model.fit(X_resampled, y_resampled)

# Predict on the test data
y_pred_test = model.predict(X_test_scaled)

# Calculate Training AUC and Test AUC
train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
test_auc = roc_auc_score(y_test, y_pred_test)

# Calculate Training Accuracy and Test Accuracy
train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
test_accuracy = accuracy_score(y_test, y_pred_test)

# Print evaluation metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("---------------------------------------------")

Training AUC: 0.7153959273707644
Test AUC: 0.7140338680578517
Training Accuracy: 0.7153959273707645
Test Accuracy: 0.702635678631609
---------------------------------------------


#### Cross Validation

In [39]:
# Loop over different values of k
for num_folds in num_folds_list:
    print(f"Performing {num_folds}-fold Cross-Validation")

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Create a model
    model = SGDClassifier(loss='log', random_state=42)

    # Perform k-fold cross-validation and get predicted probabilities
    predicted_probs = cross_val_predict(model, X_resampled, y_resampled, cv=num_folds, method='predict_proba')

    # Calculate AUC for training data
    train_auc = roc_auc_score(y_resampled, predicted_probs[:, 1])

    # Train the model on the entire training data
    model.fit(X_resampled, y_resampled)

    # Predict on the train data
    y_pred_train = model.predict(X_resampled)

    # Calculate training accuracy
    train_accuracy = accuracy_score(y_resampled, y_pred_train)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate AUC and accuracy for test data
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for each value of k
    print(f"{num_folds}-fold Cross-Validation")
    print("Training AUC:", train_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test AUC:", test_auc)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Performing 5-fold Cross-Validation
5-fold Cross-Validation
Training AUC: 0.7535386253097397
Training Accuracy: 0.7153959273707645
Test AUC: 0.7140338680578517
Test Accuracy: 0.702635678631609
---------------------------------------------
Performing 10-fold Cross-Validation
10-fold Cross-Validation
Training AUC: 0.7525790868793261
Training Accuracy: 0.7153959273707645
Test AUC: 0.7140338680578517
Test Accuracy: 0.702635678631609
---------------------------------------------
Performing 20-fold Cross-Validation
20-fold Cross-Validation
Training AUC: 0.753582098587017
Training Accuracy: 0.7153959273707645
Test AUC: 0.7140338680578517
Test Accuracy: 0.702635678631609
---------------------------------------------


#### 70/30 Split

In [40]:
# Extract the selected features and target variable
X = df.drop('PolicyIssued', axis=1)
y = df["PolicyIssued"]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Applying SMOTE to the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

# Convert the resampled arrays back to a dataframe
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df["PolicyIssued"] = y_resampled

# Train the model on the entire training data
model = SGDClassifier(loss='log', random_state=42)
model.fit(X_resampled, y_resampled)

# Predict on the test data
y_pred_test = model.predict(X_test_scaled)

# Calculate Training AUC and Test AUC
train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
test_auc = roc_auc_score(y_test, y_pred_test)

# Calculate Training Accuracy and Test Accuracy
train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
test_accuracy = accuracy_score(y_test, y_pred_test)

# Print evaluation metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("---------------------------------------------")

Training AUC: 0.7153959273707644
Test AUC: 0.7140338680578517
Training Accuracy: 0.7153959273707645
Test Accuracy: 0.702635678631609
---------------------------------------------


#### Cross Validation

In [41]:
# Loop over different values of k
for num_folds in num_folds_list:
    print(f"Performing {num_folds}-fold Cross-Validation")

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Create a model
    model = SGDClassifier(loss='log', random_state=42)

    # Perform k-fold cross-validation and get predicted probabilities
    predicted_probs = cross_val_predict(model, X_resampled, y_resampled, cv=num_folds, method='predict_proba')

    # Calculate AUC for training data
    train_auc = roc_auc_score(y_resampled, predicted_probs[:, 1])

    # Train the model on the entire training data
    model.fit(X_resampled, y_resampled)

    # Predict on the train data
    y_pred_train = model.predict(X_resampled)

    # Calculate training accuracy
    train_accuracy = accuracy_score(y_resampled, y_pred_train)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate AUC and accuracy for test data
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for each value of k
    print(f"{num_folds}-fold Cross-Validation")
    print("Training AUC:", train_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test AUC:", test_auc)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Performing 5-fold Cross-Validation
5-fold Cross-Validation
Training AUC: 0.7542858996293691
Training Accuracy: 0.7122382764174505
Test AUC: 0.7117204900291989
Test Accuracy: 0.6754276266983191
---------------------------------------------
Performing 10-fold Cross-Validation
10-fold Cross-Validation
Training AUC: 0.7531795132683377
Training Accuracy: 0.7122382764174505
Test AUC: 0.7117204900291989
Test Accuracy: 0.6754276266983191
---------------------------------------------
Performing 20-fold Cross-Validation
20-fold Cross-Validation
Training AUC: 0.7555141420227116
Training Accuracy: 0.7122382764174505
Test AUC: 0.7117204900291989
Test Accuracy: 0.6754276266983191
---------------------------------------------


### Decision Tree

#### 80/20 Split

In [43]:
# Extract the selected features and target variable
X = df.drop('PolicyIssued', axis=1)
y = df["PolicyIssued"]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Applying SMOTE to the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

# Convert the resampled arrays back to a dataframe
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df["PolicyIssued"] = y_resampled

# Train the model on the entire training data
model = DecisionTreeClassifier(random_state=42)
model.fit(X_resampled, y_resampled)

# Predict on the test data
y_pred_test = model.predict(X_test_scaled)

# Calculate Training AUC and Test AUC
train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
test_auc = roc_auc_score(y_test, y_pred_test)

# Calculate Training Accuracy and Test Accuracy
train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
test_accuracy = accuracy_score(y_test, y_pred_test)

# Print evaluation metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("---------------------------------------------")

Training AUC: 0.8993076121811238
Test AUC: 0.7015710668685584
Training Accuracy: 0.8993076121811238
Test Accuracy: 0.7410421899341875
---------------------------------------------


#### Cross Validation

In [44]:
# Loop over different values of k
for num_folds in num_folds_list:
    print(f"Performing {num_folds}-fold Cross-Validation")

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Create a model
    model = DecisionTreeClassifier(random_state=42)

    # Perform k-fold cross-validation and get predicted probabilities
    predicted_probs = cross_val_predict(model, X_resampled, y_resampled, cv=num_folds, method='predict_proba')

    # Calculate AUC for training data
    train_auc = roc_auc_score(y_resampled, predicted_probs[:, 1])

    # Train the model on the entire training data
    model.fit(X_resampled, y_resampled)

    # Predict on the train data
    y_pred_train = model.predict(X_resampled)

    # Calculate training accuracy
    train_accuracy = accuracy_score(y_resampled, y_pred_train)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate AUC and accuracy for test data
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for each value of k
    print(f"{num_folds}-fold Cross-Validation")
    print("Training AUC:", train_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test AUC:", test_auc)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Performing 5-fold Cross-Validation
5-fold Cross-Validation
Training AUC: 0.7967585302406728
Training Accuracy: 0.8993076121811238
Test AUC: 0.7015710668685584
Test Accuracy: 0.7410421899341875
---------------------------------------------
Performing 10-fold Cross-Validation
10-fold Cross-Validation
Training AUC: 0.8010165678784806
Training Accuracy: 0.8993076121811238
Test AUC: 0.7015710668685584
Test Accuracy: 0.7410421899341875
---------------------------------------------
Performing 20-fold Cross-Validation
20-fold Cross-Validation
Training AUC: 0.8037915801133695
Training Accuracy: 0.8993076121811238
Test AUC: 0.7015710668685584
Test Accuracy: 0.7410421899341875
---------------------------------------------


#### 70/30 Split

In [45]:
# Extract the selected features and target variable
X = df.drop('PolicyIssued', axis=1)
y = df["PolicyIssued"]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Applying SMOTE to the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

# Convert the resampled arrays back to a dataframe
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df["PolicyIssued"] = y_resampled

# Train the model on the entire training data
model = DecisionTreeClassifier(random_state=42)
model.fit(X_resampled, y_resampled)

# Predict on the test data
y_pred_test = model.predict(X_test_scaled)

# Calculate Training AUC and Test AUC
train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
test_auc = roc_auc_score(y_test, y_pred_test)

# Calculate Training Accuracy and Test Accuracy
train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
test_accuracy = accuracy_score(y_test, y_pred_test)

# Print evaluation metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("---------------------------------------------")

Training AUC: 0.9038771270511893
Test AUC: 0.6961195049474203
Training Accuracy: 0.9038771270511893
Test Accuracy: 0.7373195701477352
---------------------------------------------


#### Cross Validation

In [46]:
# Loop over different values of k
for num_folds in num_folds_list:
    print(f"Performing {num_folds}-fold Cross-Validation")

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Create a model
    model = DecisionTreeClassifier(random_state=42)

    # Perform k-fold cross-validation and get predicted probabilities
    predicted_probs = cross_val_predict(model, X_resampled, y_resampled, cv=num_folds, method='predict_proba')

    # Calculate AUC for training data
    train_auc = roc_auc_score(y_resampled, predicted_probs[:, 1])

    # Train the model on the entire training data
    model.fit(X_resampled, y_resampled)

    # Predict on the train data
    y_pred_train = model.predict(X_resampled)

    # Calculate training accuracy
    train_accuracy = accuracy_score(y_resampled, y_pred_train)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate AUC and accuracy for test data
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for each value of k
    print(f"{num_folds}-fold Cross-Validation")
    print("Training AUC:", train_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test AUC:", test_auc)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Performing 5-fold Cross-Validation
5-fold Cross-Validation
Training AUC: 0.796357120639948
Training Accuracy: 0.9038771270511893
Test AUC: 0.6961195049474203
Test Accuracy: 0.7373195701477352
---------------------------------------------
Performing 10-fold Cross-Validation
10-fold Cross-Validation
Training AUC: 0.8002680590114236
Training Accuracy: 0.9038771270511893
Test AUC: 0.6961195049474203
Test Accuracy: 0.7373195701477352
---------------------------------------------
Performing 20-fold Cross-Validation
20-fold Cross-Validation
Training AUC: 0.802712022693709
Training Accuracy: 0.9038771270511893
Test AUC: 0.6961195049474203
Test Accuracy: 0.7373195701477352
---------------------------------------------


### Random Forest

#### 80/20 Split

In [47]:
# Extract the selected features and target variable
X = df.drop('PolicyIssued', axis=1)
y = df["PolicyIssued"]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Applying SMOTE to the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

# Convert the resampled arrays back to a dataframe
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df["PolicyIssued"] = y_resampled

# Train the model on the entire training data
model = RandomForestClassifier(random_state=42)
model.fit(X_resampled, y_resampled)

# Predict on the test data
y_pred_test = model.predict(X_test_scaled)

# Calculate Training AUC and Test AUC
train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
test_auc = roc_auc_score(y_test, y_pred_test)

# Calculate Training Accuracy and Test Accuracy
train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
test_accuracy = accuracy_score(y_test, y_pred_test)

# Print evaluation metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("---------------------------------------------")

Training AUC: 0.8992769755519698
Test AUC: 0.7014239917884028
Training Accuracy: 0.89927697555197
Test Accuracy: 0.7492766985661146
---------------------------------------------


#### Cross Validation

In [48]:
# Loop over different values of k
for num_folds in num_folds_list:
    print(f"Performing {num_folds}-fold Cross-Validation")

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Create a model
    model = RandomForestClassifier(random_state=42)

    # Perform k-fold cross-validation and get predicted probabilities
    predicted_probs = cross_val_predict(model, X_resampled, y_resampled, cv=num_folds, method='predict_proba')

    # Calculate AUC for training data
    train_auc = roc_auc_score(y_resampled, predicted_probs[:, 1])

    # Train the model on the entire training data
    model.fit(X_resampled, y_resampled)

    # Predict on the train data
    y_pred_train = model.predict(X_resampled)

    # Calculate training accuracy
    train_accuracy = accuracy_score(y_resampled, y_pred_train)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate AUC and accuracy for test data
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for each value of k
    print(f"{num_folds}-fold Cross-Validation")
    print("Training AUC:", train_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test AUC:", test_auc)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Performing 5-fold Cross-Validation
5-fold Cross-Validation
Training AUC: 0.8569425352812364
Training Accuracy: 0.89927697555197
Test AUC: 0.7014239917884028
Test Accuracy: 0.7492766985661146
---------------------------------------------
Performing 10-fold Cross-Validation
10-fold Cross-Validation
Training AUC: 0.8596530581866255
Training Accuracy: 0.89927697555197
Test AUC: 0.7014239917884028
Test Accuracy: 0.7492766985661146
---------------------------------------------
Performing 20-fold Cross-Validation
20-fold Cross-Validation
Training AUC: 0.8609803750801357
Training Accuracy: 0.89927697555197
Test AUC: 0.7014239917884028
Test Accuracy: 0.7492766985661146
---------------------------------------------


#### 70/30 Split

In [49]:
# Extract the selected features and target variable
X = df.drop('PolicyIssued', axis=1)
y = df["PolicyIssued"]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Applying SMOTE to the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

# Convert the resampled arrays back to a dataframe
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df["PolicyIssued"] = y_resampled

# Train the model on the entire training data
model = RandomForestClassifier(random_state=42)
model.fit(X_resampled, y_resampled)

# Predict on the test data
y_pred_test = model.predict(X_test_scaled)

# Calculate Training AUC and Test AUC
train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
test_auc = roc_auc_score(y_test, y_pred_test)

# Calculate Training Accuracy and Test Accuracy
train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
test_accuracy = accuracy_score(y_test, y_pred_test)

# Print evaluation metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("---------------------------------------------")

Training AUC: 0.9038304428001214
Test AUC: 0.6955383979101245
Training Accuracy: 0.9038304428001214
Test Accuracy: 0.7466881451493249
---------------------------------------------


#### Cross Validation

In [50]:
# Loop over different values of k
for num_folds in num_folds_list:
    print(f"Performing {num_folds}-fold Cross-Validation")

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Create a model
    model = RandomForestClassifier(random_state=42)

    # Perform k-fold cross-validation and get predicted probabilities
    predicted_probs = cross_val_predict(model, X_resampled, y_resampled, cv=num_folds, method='predict_proba')

    # Calculate AUC for training data
    train_auc = roc_auc_score(y_resampled, predicted_probs[:, 1])

    # Train the model on the entire training data
    model.fit(X_resampled, y_resampled)

    # Predict on the train data
    y_pred_train = model.predict(X_resampled)

    # Calculate training accuracy
    train_accuracy = accuracy_score(y_resampled, y_pred_train)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate AUC and accuracy for test data
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for each value of k
    print(f"{num_folds}-fold Cross-Validation")
    print("Training AUC:", train_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test AUC:", test_auc)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Performing 5-fold Cross-Validation
5-fold Cross-Validation
Training AUC: 0.8587768251460959
Training Accuracy: 0.9038304428001214
Test AUC: 0.6955383979101245
Test Accuracy: 0.7466881451493249
---------------------------------------------
Performing 10-fold Cross-Validation
10-fold Cross-Validation
Training AUC: 0.8610738225485239
Training Accuracy: 0.9038304428001214
Test AUC: 0.6955383979101245
Test Accuracy: 0.7466881451493249
---------------------------------------------
Performing 20-fold Cross-Validation
20-fold Cross-Validation
Training AUC: 0.8624428413303838
Training Accuracy: 0.9038304428001214
Test AUC: 0.6955383979101245
Test Accuracy: 0.7466881451493249
---------------------------------------------


### Light GBM

#### 80/20 Split

In [51]:
# Extract the selected features and target variable
X = df.drop('PolicyIssued', axis=1)
y = df["PolicyIssued"]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Applying SMOTE to the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

# Convert the resampled arrays back to a dataframe
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df["PolicyIssued"] = y_resampled

# Train the model on the entire training data
model = lgb.LGBMClassifier(random_state=42)
model.fit(X_resampled, y_resampled)

# Predict on the test data
y_pred_test = model.predict(X_test_scaled)

# Calculate Training AUC and Test AUC
train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
test_auc = roc_auc_score(y_test, y_pred_test)

# Calculate Training Accuracy and Test Accuracy
train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
test_accuracy = accuracy_score(y_test, y_pred_test)

# Print evaluation metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("---------------------------------------------")

Training AUC: 0.7690049222850841
Test AUC: 0.7509706410415861
Training Accuracy: 0.769004922285084
Test Accuracy: 0.7002511684100086
---------------------------------------------


#### Cross Validation

In [52]:
# Loop over different values of k
for num_folds in num_folds_list:
    print(f"Performing {num_folds}-fold Cross-Validation")

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Create a model
    model = lgb.LGBMClassifier(random_state=42)

    # Perform k-fold cross-validation and get predicted probabilities
    predicted_probs = cross_val_predict(model, X_resampled, y_resampled, cv=num_folds, method='predict_proba')

    # Calculate AUC for training data
    train_auc = roc_auc_score(y_resampled, predicted_probs[:, 1])

    # Train the model on the entire training data
    model.fit(X_resampled, y_resampled)

    # Predict on the train data
    y_pred_train = model.predict(X_resampled)

    # Calculate training accuracy
    train_accuracy = accuracy_score(y_resampled, y_pred_train)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate AUC and accuracy for test data
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for each value of k
    print(f"{num_folds}-fold Cross-Validation")
    print("Training AUC:", train_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test AUC:", test_auc)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Performing 5-fold Cross-Validation
5-fold Cross-Validation
Training AUC: 0.8387354309560698
Training Accuracy: 0.769004922285084
Test AUC: 0.7509706410415861
Test Accuracy: 0.7002511684100086
---------------------------------------------
Performing 10-fold Cross-Validation
10-fold Cross-Validation
Training AUC: 0.8397023596339992
Training Accuracy: 0.769004922285084
Test AUC: 0.7509706410415861
Test Accuracy: 0.7002511684100086
---------------------------------------------
Performing 20-fold Cross-Validation
20-fold Cross-Validation
Training AUC: 0.8400414469834776
Training Accuracy: 0.769004922285084
Test AUC: 0.7509706410415861
Test Accuracy: 0.7002511684100086
---------------------------------------------


#### 70/30 Split

In [53]:
# Extract the selected features and target variable
X = df.drop('PolicyIssued', axis=1)
y = df["PolicyIssued"]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Applying SMOTE to the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

# Convert the resampled arrays back to a dataframe
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df["PolicyIssued"] = y_resampled

# Train the model on the entire training data
model = lgb.LGBMClassifier(random_state=42)
model.fit(X_resampled, y_resampled)

# Predict on the test data
y_pred_test = model.predict(X_test_scaled)

# Calculate Training AUC and Test AUC
train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
test_auc = roc_auc_score(y_test, y_pred_test)

# Calculate Training Accuracy and Test Accuracy
train_accuracy = accuracy_score(y_resampled, model.predict(X_resampled))
test_accuracy = accuracy_score(y_test, y_pred_test)

# Print evaluation metrics for training and test data
print("Training AUC:", train_auc)
print("Test AUC:", test_auc)
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("---------------------------------------------")

Training AUC: 0.7708736957587358
Test AUC: 0.7514302529716709
Training Accuracy: 0.7708736957587358
Test Accuracy: 0.7001420123359969
---------------------------------------------


#### Cross Validation

In [54]:
# Loop over different values of k
for num_folds in num_folds_list:
    print(f"Performing {num_folds}-fold Cross-Validation")

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Applying SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Create a model
    model = lgb.LGBMClassifier(random_state=42)

    # Perform k-fold cross-validation and get predicted probabilities
    predicted_probs = cross_val_predict(model, X_resampled, y_resampled, cv=num_folds, method='predict_proba')

    # Calculate AUC for training data
    train_auc = roc_auc_score(y_resampled, predicted_probs[:, 1])

    # Train the model on the entire training data
    model.fit(X_resampled, y_resampled)

    # Predict on the train data
    y_pred_train = model.predict(X_resampled)

    # Calculate training accuracy
    train_accuracy = accuracy_score(y_resampled, y_pred_train)

    # Predict on the test data
    y_pred_test = model.predict(X_test_scaled)

    # Calculate AUC and accuracy for test data
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for each value of k
    print(f"{num_folds}-fold Cross-Validation")
    print("Training AUC:", train_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test AUC:", test_auc)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Performing 5-fold Cross-Validation
5-fold Cross-Validation
Training AUC: 0.841878035690324
Training Accuracy: 0.7708736957587358
Test AUC: 0.7514302529716709
Test Accuracy: 0.7001420123359969
---------------------------------------------
Performing 10-fold Cross-Validation
10-fold Cross-Validation
Training AUC: 0.8430909023486278
Training Accuracy: 0.7708736957587358
Test AUC: 0.7514302529716709
Test Accuracy: 0.7001420123359969
---------------------------------------------
Performing 20-fold Cross-Validation
20-fold Cross-Validation
Training AUC: 0.8434349207561863
Training Accuracy: 0.7708736957587358
Test AUC: 0.7514302529716709
Test Accuracy: 0.7001420123359969
---------------------------------------------


### Neural Network

#### 80/20 Split

In [60]:
# List of epochs to iterate over
epochs_list = [10, 20, 30]
    
# Extract the selected features and target variable
X = df.drop('PolicyIssued', axis=1)
y = df["PolicyIssued"]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Applying SMOTE to the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

# Convert the resampled arrays back to a dataframe
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df["PolicyIssued"] = y_resampled

# Train the model on the entire training data using a neural network
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=len(X.columns)))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Loop over epochs
for epochs in epochs_list:
    print(f"Epochs: {epochs}")

    model.fit(X_resampled, y_resampled, epochs=epochs, batch_size=32, verbose=0)

    # Predict on the test data
    y_pred_train = np.round(model.predict(X_resampled)).astype(int)
    y_pred_test = np.round(model.predict(X_test_scaled)).astype(int)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, y_pred_train)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Epochs: 10
6121/6121 [==============================] - 4s 613us/step
Training AUC: 0.807545430595225
Test AUC: 0.7490850943381502
Training Accuracy: 0.7499183023222565
Test Accuracy: 0.692525355292023
---------------------------------------------
Epochs: 20
6121/6121 [==============================] - 4s 631us/step
Training AUC: 0.8108540715128193
Test AUC: 0.7492844451384465
Training Accuracy: 0.7511386613835501
Test Accuracy: 0.6909992687501987
---------------------------------------------
Epochs: 30
6121/6121 [==============================] - 4s 645us/step
Training AUC: 0.8130974173189593
Test AUC: 0.7490739859307098
Training Accuracy: 0.7521956250893569
Test Accuracy: 0.6963405716465838
---------------------------------------------


#### 70/30 Split

In [61]:
# List of epochs to iterate over
epochs_list = [10, 20, 30]
    
# Extract the selected features and target variable
X = df.drop('PolicyIssued', axis=1)
y = df["PolicyIssued"]

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Scaling features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Applying SMOTE to the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

# Convert the resampled arrays back to a dataframe
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df["PolicyIssued"] = y_resampled

# Train the model on the entire training data using a neural network
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=len(X.columns)))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Loop over epochs
for epochs in epochs_list:
    print(f"Epochs: {epochs}")

    model.fit(X_resampled, y_resampled, epochs=epochs, batch_size=32, verbose=0)

    # Predict on the test data
    y_pred_train = np.round(model.predict(X_resampled)).astype(int)
    y_pred_test = np.round(model.predict(X_test_scaled)).astype(int)

    # Calculate Training AUC and Test AUC
    train_auc = roc_auc_score(y_resampled, model.predict(X_resampled))
    test_auc = roc_auc_score(y_test, y_pred_test)

    # Calculate Training Accuracy and Test Accuracy
    train_accuracy = accuracy_score(y_resampled, y_pred_train)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    # Print evaluation metrics for training and test data
    print("Training AUC:", train_auc)
    print("Test AUC:", test_auc)
    print("Training Accuracy:", train_accuracy)
    print("Test Accuracy:", test_accuracy)
    print("---------------------------------------------")

Epochs: 10
5356/5356 [==============================] - 3s 637us/step
Training AUC: 0.8090879126915215
Test AUC: 0.7488053052393145
Training Accuracy: 0.7511904484022315
Test Accuracy: 0.6903071281714322
---------------------------------------------
Epochs: 20
5356/5356 [==============================] - 4s 807us/step
Training AUC: 0.8123295126438521
Test AUC: 0.7470766458104389
Training Accuracy: 0.7512663103102168
Test Accuracy: 0.7015621356959664
---------------------------------------------
Epochs: 30
5356/5356 [==============================] - 4s 812us/step
Training AUC: 0.81596127094484
Test AUC: 0.7476836415703181
Training Accuracy: 0.7531570224784669
Test Accuracy: 0.702134424214163
---------------------------------------------
